In [7]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
import argparse
import yaml
import time
import datetime
import cv2
import numpy as np
import pandas as pd
import random

from skimage import io, transform
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from scipy import ndimage, misc

import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms, utils
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch
from torch.autograd import Variable
import torch.optim as optim
from sklearn.metrics import mean_absolute_error

from google.colab import drive
import os
import warnings
from collections import defaultdict
from sklearn.model_selection import KFold
import time
from scipy import stats
import sys


In [9]:
categorical_columns = ['GENUS', 'SPECIES', 'SETTIMEOFDAY', 'TRAPTYPE']
numerical_columns = ['DIFF_DAYS', 'uvIndex13', 'uvIndex5', 'uvIndex12','uvIndex6', 'uvIndex9','precipIntensityMaxTime12','precipIntensityMaxTime14','sunriseTime12','sunsetTime1','sunsetTime4','sunsetTime6','sunriseTime13','sunsetTime7','sunsetTime3','sunsetTime14']

print(numerical_columns)

['DIFF_DAYS', 'uvIndex13', 'uvIndex5', 'uvIndex12', 'uvIndex6', 'uvIndex9', 'precipIntensityMaxTime12', 'precipIntensityMaxTime14', 'sunriseTime12', 'sunsetTime1', 'sunsetTime4', 'sunsetTime6', 'sunriseTime13', 'sunsetTime7', 'sunsetTime3', 'sunsetTime14']


In [10]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/BerkeleyResults/Approach2/aedes_mosquito_weather_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category')

categorical_column_sizes = [len(sentinel_frame[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_column_sizes]

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())
    
def get_categorical_tensor(in_frame, categories):
    categorical_data = np.stack([in_frame[col].cat.codes.values for col in categories], 1)
    categorical_data = torch.tensor(categorical_data, dtype=torch.int64)

    return categorical_data

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

In [11]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class SwishM(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [12]:
def train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr):
    model = Sentinel_net(categorical_embedding_sizes, train_n_t.shape[1], 1, dropout, hidden_size)
    model.train().cuda()

    criterion = torch.nn.L1Loss()
    params_dict = dict(model.named_parameters())
    params = []
    best = sys.maxsize
    r_value = 0
    std_err = 0
    
    for key, value in params_dict.items():
        if 'weight' in key and 'layers' in key:
            params += [{'params':value, 'weight_decay':4e-3}]
        else:
            params += [{'params':value, 'weight_decay':0.0}]

    optimizer = torch.optim.RMSprop(params, lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9, centered=False)
    epoch = 0

    for i in range(1, 1000):
        y_pred = model(train_c_t, train_n_t)
        loss = criterion(y_pred.squeeze(), outputs_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            y_pred2 = model(test_c_t, test_n_t)
            c_out, c_pred = outputs_test.cpu().detach().numpy(), y_pred2.cpu().detach().numpy().squeeze()
            current_score = mean_absolute_error(c_out, c_pred)
            if current_score < best:
                best = current_score
                m, b, r_value, p_value, std_err = stats.linregress(c_out,c_pred)

            model.train()
    
    #print('MAE = ',current_score, 'r2 = ',r_value, 'SE = ',std_err)
    return best, r_value, std_err

In [13]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('2 Layers Architecture')

for dropout in range(0,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

2 Layers Architecture
MAE =  4.679069995880127 r2 =  0.2467521771628213 SE =  0.01008776223073033 lr =  0.01 hz =  50 dr =  0.0
Best so far 4.679069995880127

MAE =  4.6784224033355715 r2 =  0.17078869678397254 SE =  0.009787385624931758 lr =  0.001 hz =  50 dr =  0.0
Best so far 4.6784224033355715

MAE =  5.547161674499511 r2 =  0.10583052736225798 SE =  0.02404309827549685 lr =  0.0001 hz =  50 dr =  0.0
MAE =  4.718043470382691 r2 =  0.1595912704686873 SE =  0.011396973921340208 lr =  0.01 hz =  60 dr =  0.0
MAE =  4.673023414611817 r2 =  0.2023079583095328 SE =  0.010850449654068051 lr =  0.001 hz =  60 dr =  0.0
Best so far 4.673023414611817

MAE =  5.443970584869385 r2 =  0.16592919490365174 SE =  0.03604144194461364 lr =  0.0001 hz =  60 dr =  0.0
MAE =  4.675738954544068 r2 =  0.18773405114347663 SE =  0.008264276788745872 lr =  0.01 hz =  70 dr =  0.0
MAE =  4.665957927703857 r2 =  0.23915726916816218 SE =  0.01344091256438443 lr =  0.001 hz =  70 dr =  0.0
Best so far 4.66595

In [14]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class SwishM(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [15]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('3 Layers Architecture')

for dropout in range(0,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

3 Layers Architecture
MAE =  4.660066175460815 r2 =  0.1927049042252611 SE =  0.013454330694016891 lr =  0.01 hz =  50 dr =  0.0
Best so far 4.660066175460815

MAE =  4.674869871139526 r2 =  0.20494373139387934 SE =  0.011351294601000378 lr =  0.001 hz =  50 dr =  0.0
MAE =  5.521320629119873 r2 =  0.09643794641830389 SE =  0.025292111597412688 lr =  0.0001 hz =  50 dr =  0.0
MAE =  4.711974096298218 r2 =  0.1834494004301574 SE =  0.012274634949693872 lr =  0.01 hz =  60 dr =  0.0
MAE =  4.666060161590576 r2 =  0.18641803327499773 SE =  0.01280028540158118 lr =  0.001 hz =  60 dr =  0.0
MAE =  5.634000682830811 r2 =  0.1153204495268028 SE =  0.024351052514629386 lr =  0.0001 hz =  60 dr =  0.0
MAE =  4.693273735046387 r2 =  0.20141132063513983 SE =  0.012134271901641224 lr =  0.01 hz =  70 dr =  0.0
MAE =  4.662367630004883 r2 =  0.2283261440224401 SE =  0.01315392375009139 lr =  0.001 hz =  70 dr =  0.0
MAE =  5.445947742462158 r2 =  0.055032896110045984 SE =  0.019768120595298578 lr 

In [16]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class SwishM(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))
    
    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [17]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('4 Layers Architecture')

for dropout in range(0,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

4 Layers Architecture
MAE =  4.603914403915406 r2 =  0.2406460370155739 SE =  0.011875327351593854 lr =  0.01 hz =  50 dr =  0.0
Best so far 4.603914403915406

MAE =  4.696462631225586 r2 =  0.16243625298739378 SE =  0.016480802475958355 lr =  0.001 hz =  50 dr =  0.0
MAE =  5.566187000274658 r2 =  0.11955359614378898 SE =  0.021498459927113665 lr =  0.0001 hz =  50 dr =  0.0
MAE =  4.677961349487305 r2 =  0.22357276974877077 SE =  0.01259798354987427 lr =  0.01 hz =  60 dr =  0.0
MAE =  4.696612215042114 r2 =  0.18610391702864723 SE =  0.014330253921291688 lr =  0.001 hz =  60 dr =  0.0
MAE =  5.412165927886963 r2 =  0.08375929588949288 SE =  0.022705350822547446 lr =  0.0001 hz =  60 dr =  0.0
MAE =  4.647363567352295 r2 =  0.17707123651384338 SE =  0.01071411519747859 lr =  0.01 hz =  70 dr =  0.0
MAE =  4.6819850444793705 r2 =  0.15294574010577278 SE =  0.015144606706498063 lr =  0.001 hz =  70 dr =  0.0
MAE =  5.449861717224121 r2 =  0.07710148945368353 SE =  0.01869733761334061 l